# BEYlink — Beirut Urban Intelligence

An AI-driven urban analysis tool for Beirut neighbourhoods. This notebook:

1. Loads Point-of-Interest (POI) data for Beirut plots from a CSV file
2. Generates radar charts for each plot's POI profile
3. Applies cluster labels to identify neighbourhood typologies
4. Creates an interactive Folium map with context-aware intervention proposals
5. Provides a Gradio web-app interface for exploring individual plots

**Running in Google Colab?** Mount your Drive and update `CSV_PATH` below.  
**Running locally?** Set `CSV_PATH` to the path of your `karim_beirut.csv` file.

In [ ]:
# Install required packages (safe to re-run)
!pip install -q pyproj folium gradio pandas matplotlib requests pillow

In [ ]:
import io
import base64
import json
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import folium
from io import BytesIO
from PIL import Image

In [ ]:
# ── Mount Google Drive if running in Colab ──────────────────────────────────
try:
    from google.colab import drive
    drive.mount('/content/drive')
    CSV_PATH = (
        '/content/drive/MyDrive/IAAC_Y2/AI_URBANISM/AI_URB_2/'
        'final_tuesday-app/Copy of karim_beirut.csv'
    )
except ImportError:
    # Running locally — update this path to your CSV file
    CSV_PATH = 'karim_beirut.csv'

poi_data = pd.read_csv(CSV_PATH)
print(f'Loaded {len(poi_data)} rows')
print(poi_data.head())

In [ ]:
POI_COLUMNS = [
    'Resort', 'Playground', 'NGO', 'Government', 'Diplomatic', 'Coworking',
    'Company', 'Viewpoint', 'Artwork', 'Restaurant', 'Pub', 'Hotel',
    'Guest House', 'Cafe', 'Worship', 'Museum', 'Theatre', 'Library',
    'Hospital', 'University', 'School', 'Fitness Centre'
]

for _, row in poi_data.iterrows():
    values = row[POI_COLUMNS].values.astype(float)
    angles = np.linspace(0, 2 * np.pi, len(POI_COLUMNS), endpoint=False)
    angles_c = np.concatenate((angles, [angles[0]]))
    values_c = np.concatenate((values, [values[0]]))

    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    ax.plot(angles_c, values_c, linewidth=1, linestyle='solid')
    ax.fill(angles_c, values_c, alpha=0.25)
    ax.set_xticks(angles)
    ax.set_xticklabels(POI_COLUMNS, fontsize=7)
    ax.set_title(f"Plot Area {row['PLOT AREA']}")
    plt.tight_layout()
    plt.show()
    plt.close(fig)

In [ ]:
CLUSTER_LABELS = {
    9: 'Leisure hub',
    8: 'Artistic and Touristic hub',
    5: 'Civic artist hub',
    4: 'Active hub',
    3: 'Unity district',
    1: 'Recreation hub'
}

poi_data['clusterID'] = poi_data['clusterID'].replace(CLUSTER_LABELS)
print(poi_data[['PLOT AREA', 'clusterID']].to_string())

In [ ]:
# All keys are lowercase for case-insensitive lookup
INTERVENTIONS = {
    'leisure hub':                ['Public Recreation', 'Pocket Park', 'Food Truck'],
    'artistic and touristic hub': ['Public Park', 'Urban Farming', 'Theatre and Cultural Events',
                                   'Seasonal Market', 'Artisanal Workshops'],
    'civic artist hub':           ['Urban Farming', 'Artisanal Workshops', 'Music Performance',
                                   'Workshop', 'Comfortable Gathering Green Space'],
    'active hub':                 ['Farming', 'Art Exhibition', 'Weekend Market', 'Live Music Performance'],
    'unity district':             ['Urban Garden', 'Urban Farming', 'Entertainment Space', 'Outdoor Gym'],
    'recreation hub':             ['Mobile Food Trucks', 'Pop-up Events', 'Art Installation']
}

BENEFITS = {
    'public recreation':               'Public recreation brings people together, improves physical and mental health, enhances safety, benefits the environment, boosts the local economy, and enhances the overall quality of life for residents.',
    'pocket park':                     'Pocket parks provide a place for residents to relax, socialize, and connect with nature. They promote physical activity, improve mental well-being, and contribute to a sense of community while enhancing neighbourhood appeal.',
    'food truck':                      'Neighbourhood food trucks provide convenient dining, support local businesses, foster community connections, offer diverse cuisine, promote sustainability, and add vibrancy to the area.',
    'public park':                     'Public parks provide a space for community gatherings, promote physical and mental well-being, enhance safety, contribute to a healthier environment, and boost property values.',
    'urban farming':                   'Urban farming provides access to fresh, locally grown produce, promotes sustainable food practices, enhances food security, beautifies the community, and encourages physical activity.',
    'theatre and cultural events':     'Theatre and cultural events promote community engagement, foster a sense of belonging, celebrate diversity, support local talent, stimulate the local economy, and enhance cultural understanding.',
    'seasonal market':                 'Seasonal markets bring fresh, local products, support local businesses, foster community connections, promote sustainability, and stimulate the local economy.',
    'artisanal workshops':             'Artisanal workshops provide opportunities for creativity and skill development, support local artisans, promote cultural preservation, encourage community participation, and contribute to the local economy.',
    'music performance':               'Music performances bring joy and entertainment to the community, foster togetherness, showcase local talent, provide cultural enrichment, and stimulate the local economy.',
    'workshop':                        'Workshops provide opportunities for learning and skill development, foster community engagement, promote personal growth, encourage creativity, and facilitate knowledge sharing.',
    'comfortable gathering green space': 'Comfortable gathering green spaces provide inviting and inclusive areas for people to gather, socialise, and relax. They foster a sense of community and promote social interactions.',
    'farming':                         'Farming activities provide access to fresh produce, promote sustainable food practices, enhance food security, create educational opportunities, and contribute to a greener neighbourhood.',
    'art exhibition':                  'Art exhibitions provide cultural enrichment, foster community engagement, support local artists, stimulate the local economy, and promote creativity and artistic expression.',
    'weekend market':                  'Weekend markets bring fresh, local products, support local businesses, foster community connections, promote sustainability, and stimulate the local economy.',
    'live music performance':          'Live music performances bring joy and entertainment to the community, foster togetherness, showcase local talent, provide cultural enrichment, and stimulate the local economy.',
    'urban garden':                    'Urban gardens provide access to fresh produce, promote sustainable food practices, beautify the community, encourage physical activity, create educational opportunities, and foster community engagement.',
    'entertainment space':             'Entertainment spaces provide a venue for community events, promote social interactions, foster a sense of belonging, support local talent, and stimulate the local economy.',
    'outdoor gym':                     'Outdoor gyms provide free access to exercise equipment, promote physical activity, enhance well-being, encourage social interactions, and create opportunities for outdoor recreation.',
    'mobile food trucks':              'Mobile food trucks provide convenient dining options, support local businesses, foster community connections, offer diverse cuisine, promote sustainability, and add vibrancy to the area.',
    'pop-up events':                   'Pop-up events bring unique and temporary experiences, foster community engagement, offer cultural enrichment, support local businesses, and stimulate the local economy.',
    'art installation':                'Art installations bring beauty and creativity to public spaces, foster community engagement, stimulate dialogue and reflection, enhance the aesthetic appeal, and support local artists.'
}

In [ ]:
beirut_map = folium.Map(
    location=[33.8938, 35.5018],
    zoom_start=13,
    control_scale=True
)
folium.TileLayer('cartodbdark_matter').add_to(beirut_map)

for index, location_info in poi_data.iterrows():
    cluster_id = str(location_info['clusterID']).lower()
    interventions_list = INTERVENTIONS.get(cluster_id, [])

    # ── Radar chart ─────────────────────────────────────────────────────────
    values = poi_data.loc[index, POI_COLUMNS].values.astype(float)
    angles = np.linspace(0, 2 * np.pi, len(POI_COLUMNS), endpoint=False).tolist()
    values_c = np.concatenate((values, [values[0]]))
    angles_c = angles + angles[:1]

    fig, ax = plt.subplots(figsize=(4, 4), subplot_kw=dict(polar=True))
    ax.plot(angles_c, values_c, color='purple', linewidth=2)
    ax.fill(angles_c, values_c, color='purple', alpha=0.25)
    ax.set_xticks(angles)
    ax.set_xticklabels(POI_COLUMNS, fontsize=6)
    ax.set_yticks([1, 2, 3, 4, 5])
    ax.set_yticklabels(['1', '2', '3', '4', '5'], color='grey', fontsize=8)
    ax.set_ylim(0, 5)
    ax.set_title(f'Cluster: {location_info["clusterID"]}', size=11, y=1.1)
    ax.grid(True)

    buf = io.BytesIO()
    plt.savefig(buf, format='png', dpi=80, bbox_inches='tight')
    plt.close(fig)
    buf.seek(0)
    chart_b64 = base64.b64encode(buf.getvalue()).decode()

    # ── Build safe benefits JSON for this marker only ────────────────────────
    local_benefits = {
        iv: BENEFITS.get(iv.lower(), 'Benefits coming soon.')
        for iv in interventions_list
    }
    benefits_json = json.dumps(local_benefits)

    options_html = '<option value="">Select an intervention</option>'
    for iv in interventions_list:
        options_html += f'<option value="{iv}">{iv}</option>'

    # ── Popup HTML ───────────────────────────────────────────────────────────
    html = f'''
    <style>
        body {{ font-family: Raleway, sans-serif; font-size: 13px; margin: 0; }}
        .lbl {{ font-weight: bold; margin-top: 8px; display: block; }}
        select {{ width: 100%; margin: 6px 0; padding: 4px; }}
        #content_{index} {{ margin-top: 6px; }}
    </style>
    <img src="{location_info['Image Url']}" alt="Site image" width="340"
         style="border-radius:4px; display:block; margin-bottom:6px">
    <span class="lbl">Cluster: {location_info['clusterID']}</span>
    <span class="lbl">Interventions:</span>
    <select id="iv_select_{index}" onchange="showContent_{index}(this)">
        {options_html}
    </select>
    <div id="content_{index}" style="display:none">
        <span class="lbl">Benefits:</span>
        <p id="benefits_text_{index}" style="margin:4px 0 8px"></p>
    </div>
    <img src="data:image/png;base64,{chart_b64}" alt="Radar chart" width="340">
    <script>
    (function() {{
        var benefits = {benefits_json};
        window.showContent_{index} = function(sel) {{
            var val = sel.value;
            var content = document.getElementById('content_{index}');
            var bText = document.getElementById('benefits_text_{index}');
            if (val) {{
                bText.innerText = benefits[val] || 'Benefits coming soon.';
                content.style.display = 'block';
            }} else {{
                content.style.display = 'none';
            }}
        }};
    }})();
    </script>
    '''

    iframe = folium.IFrame(html=html, width=380, height=520)
    popup = folium.Popup(iframe, max_width=500)

    folium.Marker(
        [location_info['Y'], location_info['X']],
        popup=popup,
        icon=folium.Icon(color='purple')
    ).add_to(beirut_map)

beirut_map

## Gradio Map App

Interactive web interface to explore individual plots on the map.

In [ ]:
import gradio as gr
from folium.raster_layers import ImageOverlay

def visualize_map(latitude, longitude, plot_image_url, plot_description):
    m = folium.Map(location=[latitude, longitude], zoom_start=14)
    folium.Marker(
        [latitude, longitude],
        popup='Selected Plot',
        icon=folium.Icon(color='purple')
    ).add_to(m)

    if plot_image_url and plot_image_url.strip().startswith('http'):
        bounds = [
            [latitude - 0.005, longitude - 0.005],
            [latitude + 0.005, longitude + 0.005]
        ]
        ImageOverlay(plot_image_url, bounds=bounds, opacity=0.6).add_to(m)

    map_html = m.get_root().render()
    desc_html = (
        f'<h3 style="font-family:sans-serif">Plot Description</h3>'
        f'<p style="font-family:sans-serif">{plot_description}</p>'
        if plot_description else ''
    )
    return map_html, desc_html

demo = gr.Interface(
    fn=visualize_map,
    inputs=[
        gr.Number(label='Latitude',    value=33.8938),
        gr.Number(label='Longitude',   value=35.5018),
        gr.Textbox(label='Plot Image URL',   placeholder='https://...'),
        gr.Textbox(label='Plot Description', lines=3,
                   placeholder='Describe this plot...')
    ],
    outputs=[
        gr.HTML(label='Map'),
        gr.HTML(label='Description')
    ],
    title='BEYlink — Plot Visualiser',
    description='Visualise a Beirut plot on an interactive map with an optional image overlay.'
)

demo.launch(share=True)